# Summer project 

Note that you should have CAMB installed, which can be done by running `pip install camb`. For more information, visit https://camb.info/.

The theory is heavily inspired in the articles contained on the folder ''Papers/''

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import sys, platform, os
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
from astropy import constants as const
import scipy.integrate as integrate
import scipy.interpolate as interpolate

#Assume installed from github using "git clone --recursive https://github.com/cmbant/CAMB.git"
#This file is then in the docs folders. Delete these two lines for pip/conda install.
camb_path = os.path.realpath(os.path.join(os.getcwd(),'..'))
sys.path.insert(0,camb_path)
import camb
from camb import model, initialpower
print('Using CAMB %s installed at %s'%(camb.__version__,os.path.dirname(camb.__file__)))
# make sure the version and path is what you expect

# matplotlib config
plt.rc('font', size=15)  #fontsize for plots
#plt.rc('axes', titlesize=20)#fontsize of the title

In [ ]:
l_speed = const.c.value / 1000

In [ ]:
dict_fiducial = {'Omegam': 0.32, 'Omegab': 0.05, 'Omegade': 0.68, 'Omegach2': 0.12055785610846023,
                 'w0': -1.0, 'wa': 0, 'hubble': 0.67, 'ns': 0.96, 'sigma8': 0.815584, 'gamma': 0.55}

locals().update(dict_fiducial)

## Cosmological parameters from Planck

From [Planck Collaboration (2018)](https://arxiv.org/pdf/1807.06209.pdf) Table 1, ''Combined'' column.

Something worth remembering $\sum_i^n \Omega_i(z) + \Omega_K(z) = 1$

In [ ]:
pars = model.CAMBparams()  # Set of parameters created

pars.set_dark_energy(w=-1.0, wa=0, dark_energy_model='fluid')
pars.set_cosmology(H0=67, ombh2=Omegab*hubble**2, omch2=Omegach2, tau=0.058);
pars.InitPower.set_params(As=2.1260500000000005e-9, ns=ns);

In [ ]:
results = camb.get_results(pars)

omk = pars.omk
omm = 1 - Omegade - omk

## Background Quantities

In the context of GR, cosmic acceleration requires a fluid, dubbed **Dark Energy** (DE), with an equation of state $w_{DE} < -1/3$. The standard model of csomology, commonly refered to as the $\Lambda CDM$ model, assumes that this phenomenon is due to the presence of a cosmological constant $\Lambda$ described by a constant equation state $w_{\Lambda} = -1$, which corresponds to a time-independent energy density. This has some problems (*cosmological constant problem* and *coincidence problem*), so a more general scenario for the component responsible for cosmic acceleration postulates a dynamical DE, with a dependent equation of state parameter $w_{DE}(z)$.
$$w_{DE}(z) = w_0 + w_a\frac{z}{1+z}$$
Where $w_0$ is  the present value of the equation of state and $w_a$ is a measure of its variation. The evolution of the DE density obeys
$$\rho_{DE} (z) = \rho_{DE,0} (1+z)^{3(1+w_0+w_a)}exp\left[-3w_a\frac{z}{1+z}\right] $$

Then, we can get
$$E(z) = \sqrt{
    \Omega_{m,0}(1+z)^3 + \Omega_{DE,0}(1+z)^{3(1+w_0+w_a)}exp\left[-3w_a\frac{z}{1+z}\right] + \Omega_{K,0}(1+z)^2
}$$
With the current DE density $\Omega_{DE,0}$ satisfying the relation $\Omega_{DE,0} = 1 - \Omega_{m,0} - \Omega_{K, 0}$. The $\Lambda CDM$ model can be recovered by setting $w_0=-1$ and $w_a=0$, in which case

$$E(z) = \sqrt{
    \Omega_{m,0}(1+z)^3 + \Omega_{\Lambda,0} + \Omega_{K,0}(1+z)^2
}$$

In [ ]:
def E(z, fiducial=dict_fiducial):
    locals().update(fiducial)
    mat = Omegam * (1 + z) ** 3
    de = Omegade * (1 + z) ** (3 * (1 + w0 + wa)) * \
        np.exp(-3 * wa * z / (1 + z))
    cur = (1 - Omegam - Omegade) * (1 + z) ** 2
    return np.sqrt(mat + de + cur)


In [ ]:
z_to_plot = np.linspace(0, 2.5, 100)

fig, ax = plt.subplots()

ax.plot(z_to_plot, E(z_to_plot), c='mediumpurple')
ax.set_xlabel('Redshift $z$')
ax.set_ylabel('$E(z)$');
# ax.set_title('');

## Distance measurements

The comoving distance to an object at redshift $z$ can be computed as
$$r(z)  \frac{c}{H_0} \int _0^z \frac{dz}{E(z)}$$
This quantity is not a direct observable, but it is closely related to other distance definitions that are directly linked with cosmological observations. A relevant distance is the **angular diameter disantance** (at least for our forecasts), whose definitiion is based on the relation between the apparent angular size of an object and its true physical size in Euclidean space, which is relate to the comoving distance by
$$D_A(z) = \begin{cases}
(1 + z)^{-1} \frac{c}{H_0} \frac{1}{\sqrt{|\Omega_{K,0}|}} sin\left[\sqrt{|\Omega_{K,0}|} \frac{H_0}{c}r(z)\right]& \text{if}\, \, \Omega_{K,0} < 0 \\
(1 + z)^{-1} r(z) & \text{if}\, \, \Omega_{K,0} = 0 \\
(1 + z)^{-1} \frac{c}{H_0} \frac{1}{\sqrt{\Omega_{K,0}}} sinh\left[\sqrt{\Omega_{K,0}} \frac{H_0}{c}r(z)\right]& \text{if}\, \, \Omega_{K,0} < 0 \\
\end{cases}$$

and this angular diameter distance is related to the transverse comoving distance by $D_A = D_M / (1+z)$

In [ ]:
def r(z, fiducial=dict_fiducial):
    """Comoving Distance as a function of the redshift
    trapz from numpy is used.
    ===================================================
    Input: redshift z
    Output: comoving distance r
    """
    def integrand(u):
        return 1/E(u, fiducial)
    if type(z) == np.ndarray:
        integral = np.zeros(len(z))
        for idx, redshift in enumerate(z):
            z_int = np.linspace(0, redshift, len(z))
            integral[idx] = np.trapz(integrand(z_int), z_int)
    else: 
        z_int = np.linspace(0, z, 200)
        integral = np.trapz(integrand(z_int), z_int)
    return l_speed / pars.H0 * integral


In [ ]:
z_to_plot = np.linspace(0.001, 2.5, 200)

fig, ax = plt.subplots()
ax.plot(z_to_plot, r(z_to_plot), c='mediumpurple')
ax.set_xlabel('Redshift $z$')
ax.set_ylabel('$r(z)$');
# ax.set_title('');

In [ ]:
def D_A(z, Omk=omk):
    """Angular diameter distance as a function of the redshift.
    Note that there are different definitions for different
    space-time curvatures.
    ===================================================
    Inputs: redshift z; curvature density Omk, default given by camb (optional)
    Outputs: angular diameter distance D_A
    """
    if Omk == 0:
        return r(z) / (1 + z)
    
    arg = np.sqrt(np.abs(Omk)) * pars.H0 / l_speed * r(z)
    prop_const = l_speed / pars.H0 / np.sqrt(np.abs(Omk))
    
    if Omk < 0:
        return prop_const / (1 + z) * np.sin(arg)

    else:
        return prop_const / (1 + z) * np.sinh(arg)


In [ ]:
z_to_plot = np.linspace(0.001, 2.5, 200)

fig, ax = plt.subplots()

ax.plot(z_to_plot, D_A(z_to_plot), c='mediumpurple')
ax.set_xlabel('Redshift $z$')
ax.set_ylabel('$D_A(z)$');
# ax.set_title('');

## Weak lensing observable

We refer to *weak lensing* as the physical phenomenom and *cosmic shear* as the observable summary statistic that uses weak lensing to extract cosmological parameters.

The large-sacale cosmic structure deflects the path of photons from distant galaxies, which induces distortions in the images of these galaxies. *Convergence* is a change in the observed size of a galaxy and *shear* is a change in the observed third flattening or third eccentricity (known as ellipticity or polarisation). The shear field caused by large-scale structure has zero mean because we assume isotropy and homogeneity of the Universe, but tis two-point correlation function and its power spectrum contain cosmological information that probes both the background evolution of the Universe and the growth of cosmic structure. This two point statistics are known as 'cosmic shear'.

In order to exploit the cosmic shear for cosmological parameter inference, the real/configuration space measurementof the two-point statistic, as a function of angular separation on the celestial sphere, is known as the *cosmic shear correlation function*. While the angular speherical-harmonic measurement of the two-point statistic is kown as the *cosmic shear power spectrum*. Both these statistics can be computed in a series of redshift bins, which is known as *tomography* and is required in order to achieve high-precision dark energy measurements. We will be focused on the latter statistic.

We consider five main quatities that must be modeled in order to recover the observable cosmic shear power spectrum.

1. **The cosmic shear power spectrum**. Is the change in the ellipticity of the image of a background galaxy, caused by the lensing effect of large-scale structure along the los. For an individual galaxy we express this, to linear order, as $$\epsilon = \gamma + \epsilon^I$$ where $\gamma$ is the cosmological shear and $\epsilon^I$ is the intrinsic ellipticity. This ellipticity is a spin-2 quantity with zero mean over a large survey area and a a non-zero two-point correlation function or power spectrum, encoding information on the expansion history of the Universe and the matter power spectrum. The spherical harmonic transform of the two-point correlation function is the angular power spectrum. Since the full computation of the csomic shear power spectrum is laborious, the Limber approximation is adopted, which allows us to write the observable as $$C_{ij}^{\gamma\gamma}(\ell)\simeq \frac{c}{H_0}\int dz\frac{W_i^\gamma(z)W_j^\gamma(z)}{E(z)r²(z)}P_{\delta\delta}\left[\frac{\ell + 1/2}{r(z)},z\right]$$ where $i$ and $j$ identify pairs of redshift bins, $E(z)$ was defined previously, $r(z)$ is the comoving distance, $P_{\delta\delta}(k,z)$ is the matter power spectrum evaluated at $k=k_\ell(z)\equiv(\ell+1/2)/r(z)$ (Limber approximation), and the weight functions $W^\gamma(z)$ are defined as $$W_i^\gamma(z) = \frac{3}{2}\left(\frac{H_0}{c}\right)²\Omega_{m,0}(1+z)r(z)\int_z^{z_{max}}dz'n_i(z')\left[1-\frac{\tilde{r}(z)}{\tilde{r}(z')}\right] = \frac{3}{2}\left(\frac{H_0}{c}\right)²\Omega_{m,0}(1+z)r(z)\tilde{W}_i(z)$$ where $\tilde{W}_i(z)$ is the **window function**

2. **The intrinsic alignment power spectrum** (to be continued)

In [ ]:
z_equipop_bins = [(0.001, 0.42), (0.42, 0.56), (0.56, 0.68), (0.68, 0.79), (0.79, 0.90),
                    (0.90, 1.02), (1.02, 1.15) ,(1.15, 1.32), (1.32, 1.58), (1.58, 2.50)]

# How many points fall in each bin
# [sum([(value >= low) and (value < high) for value in data]) for low, high in bins]

In [ ]:
def pdf(zp, z, cb=1.0, zb=0.0, sigb=0.05, co=1.0, zo=0.1, sigo=0.05, fout=0.1):
    """Probability distribution function.
    Describes the probability that a galaxy with
    redshift z has a measured redshift zp.
    ===================================================
    Inputs: measured reshift zp; galaxy redshift z;
    parameters to describe  the photometric redshift
    distribution of sources (optionals), defaults:
    cb=1, zb=0, sigb=0.05, co=1, zo=1, sigo=0.05, fout=0.1
    Output: probability that the galaxy of redshift z
    has a measured redshift zp.
    """
    exp1 = np.exp((-1 / 2) * ((z - cb * zp - zb) / (sigb * (1 + z))) ** 2)
    exp2 = np.exp((-1 / 2) * ((z - co * zp - zo) / (sigo * (1 + z))) ** 2)

    term1 = (1 - fout) / (np.sqrt(2 * np.pi) * sigb * (1 + z))
    term2 = fout / (np.sqrt(2 * np.pi) * sigo * (1 + z))
    return term1 * exp1 + term2 * exp2


def n(z, zm=0.9):
    """Underlying distribution of density.
    This expression is chosen in agreement with Euclid
    Red Book. Note that this does NOT take into account
    the proportional constant.
    ===================================================
    Inputs: redshift z, median redshift zp (optional,
    default is 0.9 according to Euclid Red Book)
    Output: number density at redshift z.
    """
    z0 = zm / np.sqrt(2)
    return (z / z0) ** 2 * np.exp(-(z / z0) ** (3 / 2))


def bin_number_density(i, z, zmin=0.0, zmax=2.5):
    """Number density of observed galaxies in the ith bin.
    Quad integration from scipy is used.
    ===================================================
    Inputs: bin number i, redshift z, min/max redshift 
    zmin/zmax (optional, default=0.0/2.5)
    Output: number density at redshift z in the ith bin.
    """
    ith_bin = z_equipop_bins[i]
    zi_l, zi_u = ith_bin

    def integrand1(u):
        return n(z) * pdf(u, z)    
    
    def integrand2(v, w):
        return n(w) * pdf(v, w)  

    num = integrate.quad(integrand1, zi_l, zi_u)[0]
    den = integrate.nquad(integrand2, [[zi_l, zi_u], [zmin, zmax]])[0]
    return num / den

In [ ]:
## Since the following its taking a lot of time, number density per bin will be stored
## This takes about an hour (63 min)
#
## Rounding to avoid getting numbers like 0.200000000001
#z_list = [round(i*0.001, 3) for i in range(1, 2502)]
#
#for i in range(len(z_equipop_bins)):
#    list = []
#    for z in z_list:
#        list.append([z, bin_number_density(i, z)])
#    np.savetxt('romb_bin_ndensity/bin_%s'%(str(i)), np.array(list))

In [ ]:
dict_ndsty = {}

In [ ]:
# Interpolating the bin_number_density
for i in range(10):
    dict_ndsty['ndensity_file%s'%(str(i))] = np.loadtxt('bin_ndensity/bin_%s'%(str(i)))
    dict_ndsty['bin_ndensity_%s'%(str(i))] = interpolate.interp1d(dict_ndsty['ndensity_file%s'%(str(i))][:,0], dict_ndsty['ndensity_file%s'%(str(i))][:,1])

In [ ]:
fig, ax = plt.subplots()

for i in range(10):
    ax.plot(z_to_plot, dict_ndsty['bin_ndensity_%s'%(str(i))](z_to_plot), c='mediumpurple')
ax.plot(z_to_plot, 25*n(z_to_plot), c='red')
ax.set_xlabel('Redshift $z$')
ax.set_ylabel('Number density');

In [ ]:
def window(i, z, zmax=2.5, fiducial=dict_fiducial):
    """Reduced window function as a function of redshift.
    trapz from numpy is used for integration.
    ===================================================
    Inputs: bin number i, redshift z, min/max redshift 
    zmin/zmax (optional, default=0.001/2.5)
    Output: window function evaluated at z, of the ith bin.
    """
    def integrand(w):
        return dict_ndsty['bin_ndensity_%s' % (str(i))](w) * (1 - r(z, fiducial) / r(w, fiducial))

    z_int = np.linspace(z, zmax, 200)
    integral = np.trapz(integrand(z_int), z_int)

    return integral


def weight(i, z, fiducial=dict_fiducial):
    """Weight function of the ith bin, as a function of
    redshift.
    ===================================================
    Inputs: bin number i, redshift z, matter density 
    Omega_m (optional, default provided by CAMB)
    Output: weight function evaluated at z, of the ith bin
    """
    H0 = fiducial['hubble'] * 100
    constant = 3 / 2 * (H0 / l_speed) ** 2 * fiducial['Omegam']
    z_dep = (1 + z) * r(z, fiducial) * window(i, z, fiducial=fiducial)
    return constant * z_dep

In [ ]:
i = 9

fig, ax = plt.subplots()

for z in z_to_plot:
    ax.scatter(z, window(i, z), c='mediumpurple', s=0.5)
ax.set_xlabel('Redshift $z$')
ax.set_ylabel(r'Window function $\tilde{W}_%s^{\gamma\gamma}(z)$'%(str(i)));

### Cosmic shear

In [ ]:
#Now get matter power spectra and sigma8 at redshift 0 and 0.8
pars = model.CAMBparams()  # Set of parameters created
pars.set_dark_energy(w=-1.0, wa=0, dark_energy_model='fluid')
pars.set_cosmology(H0=67, ombh2=Omegab*hubble**2, omch2=Omegach2, tau=0.058);
pars.InitPower.set_params(As=2.1260500000000005e-9, ns=ns)
#Note non-linear corrections couples to smaller scales than you want
pars.set_matter_power(redshifts=np.linspace(0.001, 2.5, 101), kmax=50.0)

#Linear spectra
pars.NonLinear = model.NonLinear_none
results = camb.get_results(pars)
kh, z, pk = results.get_matter_power_spectrum(minkh=1e-4, maxkh=50, npoints = 200)

In [ ]:
# Store data for this model
dict_mps_data = {'kh': kh, 'z': z, 'pk':pk}

for rslt in dict_mps_data:
    np.savetxt('mps_data/'+rslt, dict_mps_data[rslt])

In [ ]:
def plot_pk(zs):
    """Function for plotting the mater power spectrum
    ===================================================
    Input: list containing 2 sucesive redshift values, zs
    Output: plot
    """
    for i, redshift in enumerate(zs):
        plt.loglog(kh, pk[i,:])
    plt.xlabel('k/h Mpc');
    plt.legend(['z=%f'%zs[0],'z=%f'%zs[1]]);
    plt.title('Matter power at z=%s and z= %s'%tuple(zs));

In [ ]:
plot_pk([0.0, 0.025])

In [ ]:
# Loading data from matter power spectrum
for rslt in dict_mps_data:
    rslt = np.loadtxt('mps_data/'+rslt)

In [ ]:
# Interpolating the data
mps_linear = interpolate.RectBivariateSpline(z, kh, pk)

In [ ]:
fig, ax = plt.subplots()

ax.plot(kh, pk[0,:], 'x', color='k', label='CAMB points', ms=4)
ax.plot(kh, mps_linear(0, kh)[0], label='2D interpolation')
ax.set_yscale('log')
ax.set_xscale('log')
ax.legend();

In [ ]:
def convergence(i, j, l, fiducial=dict_fiducial, zmin=0.001, zmax=2.5):
    H0 = fiducial['hubble'] * 100
    def integrand(z):
        term1 = weight(i, z, fiducial) * weight(j, z, fiducial) / \
            (E(z, fiducial) * r(z, fiducial) ** 2)
        k = (l + 1/2) / r(z)
        term2 = mps_linear(z, k)[0]
        return term1 * term2
    integral = integrate.quad(integrand, zmin, zmax)[0]
    return l_speed / H0 * integral

In [ ]:
load_cosmic = np.loadtxt('convergence/cosmic_shear_array')

cosmic_shear_array = np.reshape(load_cosmic, (load_cosmic.shape[0], load_cosmic.shape[1] // 10, 10))

In [ ]:
fig, ax = plt.subplots()

ls = np.arange(100, 1501)
i, j = 9, 9

for l in ls:
    ax.scatter(l, l*(l+1)*cosmic_shear_array[l-100, i, j]/2/np.pi, c='mediumpurple', s=0.5)
ax.set_xlim((100, 300))
ax.set_xlabel('Multipole $\ell$')
ax.set_ylabel(r'$C_{%s%s}^{\gamma\gamma}(\ell)$'%(str(i),str(j)));
#ax.legend()

#### Correct ells to evaluate

In [ ]:
ls = np.logspace(1, np.log10(1500), 101)
l_bins = [(ls[i], ls[i + 1]) for i in range(100)]
ls_eval = np.array([(l_bins[i][1] + l_bins[i][0]) / 2 for i in range(100)])

In [46]:
# Getting the correct array
cosmic_shear = np.zeros((len(ls_eval), 10, 10))

for idx, ell in enumerate(ls_eval):
    for i in range(10):
        for j in range(10):
            cosmic_shear[idx, i, j] = convergence(i, j, ell)
            print('ell: %f'% (ell), end= '\r')

In [ ]:
reshape_cosmic = np.reshape(cosmic_shear, (cosmic_shear.shape[0], -1))
np.savetxt('convergence/cosmic_shear_correctls', reshape_cosmic)

In [ ]:
load_cosmic = np.loadtxt('convergence/cosmic_shear_array')
cosmic_shear = np.reshape(load_cosmic, (load_cosmic.shape[0], load_cosmic.shape[1] // 10, 10))

## Fisher Matrix

The error on the observed cosmic shear angular power spectrum can be expressed as $$\Delta C_{ij}^{\epsilon\epsilon}(\ell) = \sqrt{\frac{2}{(2l+1)\Delta \ell f_{sky}}}C_{ij}^{\epsilon\epsilon}(\ell)$$ where $f_{sky}$ is the fraction of surveyed sky, $\Delta \ell $ is the multipole bandwth and the last term is a Poisson noise term.

The covariance matrix is defined as $$Cov[C_{ij}^{\epsilon\epsilon}(\ell), C_{kl}^{\epsilon\epsilon}(\ell')]=\frac{C_{ik}^{\epsilon\epsilon} C_{jl}^{\epsilon\epsilon}(\ell') + C_{il}^{\epsilon\epsilon}(\ell)C_{lk}^{\epsilon\epsilon}(\ell')}{(2l+1)f_{sky}\Delta \ell}\delta_{ll'}^K$$

In the case that one assumes the signal is the covariance, the Fisher matrix reads
$$F_{\alpha\beta}=\sum_{\ell=\ell_{min}}^{\ell_{max}}\sum_{ij,mn}\frac{\partial C_{ij}^{\epsilon\epsilon}(\ell)}{\partial \theta_\alpha}\left\{[\Delta C^{\epsilon\epsilon}(\ell)]^{-1}\right\}_{jm}
\frac{\partial C_{mn}^{\epsilon\epsilon}(\ell)}{\partial \theta_\beta}\left\{[\Delta C^{\epsilon\epsilon}(\ell)]^{-1}\right\}_{ni}$$ No cross correlation between the modes with different $l_s$ is included, which is true under the Gaussian covariance assumption.

In the case that one assumes the signal is the mean power spectrum, the Fisher matrix reads
$$F_{\alpha\beta}=\sum_{\ell=\ell_{min}}^{\ell{max}}\sum_{ij,mn}\frac{\partial C_{ij}^{\epsilon\epsilon}(\ell)}{\partial \theta_\alpha}\text{Cov}^{-1}[C_{ij}^{\epsilon\epsilon}(\ell), C_{mn}^{\epsilon\epsilon}(\ell)]
\frac{\partial C_{mn}^{\epsilon\epsilon}(\ell)}{\partial \theta_\beta}$$ 

The derivative, can be computed as follows. Since the cosmic shear is the dominant observable, the others are negleclted, ie, $C_{ij}^{\epsilon\epsilon} \approx C_{ij}^{\gamma\gamma} + N_{ij}^\epsilon(\ell)$. Then, the derivated of interest is
$$\frac{\partial C_{ij}^{\gamma\gamma}(\ell)}{\partial \theta_\mu} = \int_{z_{min}}^{z_{max}}dz\frac{\partial K_{ij}^{\gamma\gamma}(z)}{\partial \theta_\mu} P_{\delta\delta}[k_\ell(z),z] +\int_{z_{min}}^{z_{max}}dzK_{ij}^{\gamma\gamma}(z)\frac{\partial P_{\delta\delta}[k_\ell(z),z]}{\partial \theta_\mu}$$ 

where 
$K_{ij}^{\gamma\gamma}(z) =  \left[\frac{3}{2}\Omega_{m,0}(1+z)\right]²\left(\frac{H_0}{c}\right)³\frac{\tilde{W}_i^\gamma(z)\tilde{W}_j^\gamma(z)}{E(z)}$

In [ ]:
def error_convergence(i, j, l, fsky=1.0, dl=10):
    term1 = np.sqrt(2 / ((2 * l + 1) * dl * fsky))
    term2 = cosmic_shear[l]
    return term1 * term2

In [ ]:
ell = 10

fig, ax = plt.subplots()

im = ax.imshow(error_convergence(ell - 10))
ax.set_title('$\Delta C_{ij}^{\gamma\gamma}(\ell)$')
ax.set_ylim(ax.get_ylim()[::-1])
fig.colorbar(im)

In [ ]:
def covariance(i, j, m, n, fsky=(1/15000), dl=1490):
    covariance_matrix = np.zeros((100, 100))
    for idx, l in enumerate(ls_eval):
        num1 = cosmic_shear_array[int(np.round(l, 0)), i, m] * cosmic_shear_array[int(np.round(l, 0)), j, n]
        num2 = cosmic_shear_array[int(np.round(l, 0)), i, n] * cosmic_shear_array[int(np.round(l, 0)), j, m]
        covariance_matrix[idx] = (num1 + num2) / ((2 * l + 1) * fsky * dl)
    return covariance_matrix